In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy
import tqdm


In [ ]:
def read_train_data():
    train_df = pd.read_csv('../input/shopee-product-matching/train.csv')
    train_df['posting_id'] = train_df['posting_id'].apply(lambda x: int(x.split('_')[1]))
    return train_df[['posting_id', 'image_phash', 'label_group']]

In [ ]:
train = read_train_data()
train.sort_values('image_phash').head()

In [ ]:
!pip install ImageHash

In [ ]:
import imagehash

In [ ]:
train['binhash'] = train['image_phash'].apply(lambda x: imagehash.hex_to_hash(x).hash.flatten().astype(np.int0))

In [ ]:
train

In [ ]:
import math

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
test_df = pd.read_csv('../input/shopee-product-matching/test.csv')
#test_df['posting_id'] = test_df['posting_id'].apply(lambda x: int(x.split('_')[1]))
test_df = test_df[['posting_id', 'image_phash']]

In [ ]:
hashes = test_df['image_phash'].apply(lambda x: imagehash.hex_to_hash(x).hash.flatten().astype(np.int0)).apply(pd.Series)

In [ ]:
from sklearn.manifold import SpectralEmbedding
embedding = SpectralEmbedding(n_components=1, n_neighbors=10, n_jobs=-1)

In [ ]:
hashes = embedding.fit_transform(hashes)

In [ ]:
len(hashes)

In [ ]:
labels = DBSCAN(eps=8.910612167916213e-05, min_samples=2, n_jobs=-1).fit_predict(hashes)

In [ ]:
test_df['labels'] = labels

In [ ]:
def make_submission_df(df):
    results = []
    for label, d in df.groupby('labels'):
        if label != -1:
            row = ' '.join(d['posting_id'].values)
            results.append(pd.DataFrame({
                'matches': row
                for _ in d['posting_id'].values
            }, index=d['posting_id'].values))
        else:
            results.append(
                pd.DataFrame(
                    { 'matches': d['posting_id'].values},
                    index=d['posting_id'].values
                )
            )
    res = pd.concat(results)
    res.index.name='posting_id'
    return res

In [ ]:
make_submission_df(test_df).to_csv('submission.csv')